# Experiments
## Online linear regression with absolute loss

This notebook contains a experiment script for reproducing experimental results of the paper ``Parameter-free Online Linear Optimization with Side Information via Universal Coin Betting''.
For each dataset `['BeijingPM2pt5', 'MetroInterstateTrafficVolume']`, run this notebook and run `generate-plots.ipynb`.

In [ ]:
from collections import defaultdict
from timeit import default_timer as timer
import numpy as np

import datasets
from learners import \
    MarkovSideInformation, \
    OnlineGradientDescent, \
    DimensionFreeExponentiatedGradient, AdaptiveNormal, \
    KT, ContextTreeWeighting, \
    Addition, Mixture
from problems import LinearRegressionWithAbsoluteLoss
from quantizer import Quantizer, get_standard_vector

%load_ext autoreload
%autoreload 2

In [ ]:
# Pick one of ['BeijingPM2pt5', 'MetroInterstateTrafficVolume']
dataset_name = 'BeijingPM2pt5'
assert dataset_name in ['BeijingPM2pt5', 'MetroInterstateTrafficVolume']
Dataset = getattr(datasets, dataset_name)

In [ ]:
problem = LinearRegressionWithAbsoluteLoss()

dataset = Dataset(
    root='../',
    standardize=True,
    rescale=False,
    bias=True,
    normalize=True,
    batch_normalize=False)
dim = dataset.X.shape[1]

# data
X, y = dataset.X, dataset.y
data = X, y

In [ ]:
X.shape

In [ ]:
depths_markov = [0] + list(range(1, 13, 2))

In [ ]:
# OGD with Markov side information
ogds = defaultdict(lambda: defaultdict(dict))
lr_scales = np.array([1.5 ** n for n in np.arange(5, 25)])
for i in range(dim):
    for depth in depths_markov:
        for lr_scale in lr_scales:
            ogds[i][depth][lr_scale] = OnlineGradientDescent(
                dim=dim,
                lr_scale=lr_scale,
                problem=problem,
                side_information=MarkovSideInformation(
                    depth=depth,
                    quantizer=Quantizer(quantizer_vector=get_standard_vector(dim, i)),
                ),
            ).fit(data)
        print(depth, end=' ')
    print()


In [ ]:
init_wealth = 10 ** (-5.)

# with Markov side information
kt_markovs = defaultdict(dict)
dfeg_markovs = defaultdict(dict)
adanorm_markovs = defaultdict(dict)
for i in range(dim):
    for depth in depths_markov:
        dfeg_markovs[i][depth] = DimensionFreeExponentiatedGradient(
            dim=dim,
            problem=problem,
            side_information=MarkovSideInformation(
                depth,
                quantizer=Quantizer(quantizer_vector=get_standard_vector(dim, i)))
        ).fit(data)

        adanorm_markovs[i][depth] = AdaptiveNormal(
            dim=dim,
            problem=problem,
            side_information=MarkovSideInformation(
                depth,
                quantizer=Quantizer(quantizer_vector=get_standard_vector(dim, i)))
        ).fit(data)

        kt_markovs[i][depth] = KT(
            dim=dim,
            init_wealth=init_wealth,
            problem=problem,
            side_information=MarkovSideInformation(
                depth,
                quantizer=Quantizer(quantizer_vector=get_standard_vector(dim, i)))
        ).fit(data)
        print(depth, end=' ')
    print()

In [ ]:
# with CTW
times = []
ctws = defaultdict(dict)
max_depths_ctw = range(1, 13, 2)
for i in range(dim):
    print("quantizer {}".format(i))
    for max_depth in max_depths_ctw:
        start = timer()
        ctws[i][max_depth] = ContextTreeWeighting(
            dim=dim,
            init_wealth=init_wealth,
            max_depth=max_depth,
            problem=problem,
            quantizer=Quantizer(quantizer_vector=get_standard_vector(dim, i)),
        ).fit(data)
        times.append(timer() - start)
        print(max_depth, times[-1])


## Combine algorithms

In [ ]:
# combining Markovs over dimensions
depths_combine_markovs_dims = range(1, 13, 2)
add_markovs_over_dims = dict()
mix_markovs_over_dims = dict()
for depth in depths_combine_markovs_dims:
    algorithms = [
        KT(
            dim=dim,
            init_wealth=init_wealth,
            problem=problem,
            side_information=MarkovSideInformation(
                depth,
                quantizer=Quantizer(quantizer_vector=get_standard_vector(dim, i)))
        )
        for i in range(dim)]
    add_markovs_over_dims[depth] = Addition(dim=dim, algorithms=algorithms).fit(data)
    mix_markovs_over_dims[depth] = Mixture(dim, algorithms, init_wealth=init_wealth, weights=None).fit(data)
    print(depth, end=' ')

In [ ]:
# combining Markovs over depths and dimensions
max_depths_combine_markovs_depths_dims = range(1, 13, 2)
add_markovs_over_depths_dims = dict()
mix_markovs_over_depths_dims = dict()
for depth in max_depths_combine_markovs_depths_dims:
    algorithms = [
        KT(
            dim=dim,
            init_wealth=init_wealth,
            problem=problem,
            side_information=MarkovSideInformation(
                depth,
                quantizer=Quantizer(quantizer_vector=get_standard_vector(dim, i)))
        )
        for d in range(depth) for i in range(dim)]
    add_markovs_over_depths_dims[depth] = Addition(dim, algorithms).fit(data)
    mix_markovs_over_depths_dims[depth] = Mixture(dim, algorithms, init_wealth=init_wealth, weights=None).fit(data)
    print(depth, end=' ')

In [ ]:
# combining CTWs over dimensions
max_depths_combine_ctws_dims = range(1, 13, 2)
add_ctws_over_dims = dict()
mix_ctws_over_dims = dict()

for max_depth in max_depths_combine_ctws_dims:
    algorithms = [
        ContextTreeWeighting(
            dim=dim,
            init_wealth=init_wealth,
            quantizer=Quantizer(get_standard_vector(dim, i)),
            max_depth=max_depth,
            problem=problem)
        for i in range(dim)]
    add_ctws_over_dims[max_depth] = Addition(dim, algorithms).fit(data)
    mix_ctws_over_dims[max_depth] = Mixture(dim, algorithms, init_wealth=init_wealth, weights=None).fit(data)
    print(max_depth, end=' ')

## Save variables

In [ ]:
dataset_name = dataset.name

In [ ]:
ogds_cum_loss = dict()
for dim in ogds.keys():
    ogds_cum_loss[dim] = dict()
    for depth in ogds[dim].keys():
        ogds_cum_loss[dim][depth] = dict()
        for lr_scale in ogds[dim][depth].keys():
            ogds_cum_loss[dim][depth][lr_scale] = ogds[dim][depth][lr_scale].cum_loss

In [ ]:
kt_markovs_cum_loss = dict()
for dim in kt_markovs.keys():
    kt_markovs_cum_loss[dim] = dict()
    for depth in kt_markovs[dim].keys():
        kt_markovs_cum_loss[dim][depth] = kt_markovs[dim][depth].cum_loss

dfeg_markovs_cum_loss = dict()
for dim in dfeg_markovs.keys():
    dfeg_markovs_cum_loss[dim] = dict()
    for depth in dfeg_markovs[dim].keys():
        dfeg_markovs_cum_loss[dim][depth] = dfeg_markovs[dim][depth].cum_loss

adanorm_markovs_cum_loss = dict()
for dim in adanorm_markovs.keys():
    adanorm_markovs_cum_loss[dim] = dict()
    for depth in adanorm_markovs[dim].keys():
        adanorm_markovs_cum_loss[dim][depth] = adanorm_markovs[dim][depth].cum_loss

ctws_cum_loss = dict()
for dim in ctws.keys():
    ctws_cum_loss[dim] = dict()
    for depth in ctws[dim].keys():
        ctws_cum_loss[dim][depth] = ctws[dim][depth].cum_loss

In [ ]:
add_markovs_over_dims_cum_loss = dict()
for depth in add_markovs_over_dims.keys():
    add_markovs_over_dims_cum_loss[depth] = add_markovs_over_dims[depth].cum_loss

mix_markovs_over_dims_cum_loss = dict()
for depth in mix_markovs_over_dims.keys():
    mix_markovs_over_dims_cum_loss[depth] = mix_markovs_over_dims[depth].cum_loss

add_markovs_over_depths_dims_cum_loss = dict()
for depth in add_markovs_over_dims.keys():
    add_markovs_over_depths_dims_cum_loss[depth] = add_markovs_over_depths_dims[depth].cum_loss

mix_markovs_over_depths_dims_cum_loss = dict()
for depth in mix_markovs_over_dims.keys():
    mix_markovs_over_depths_dims_cum_loss[depth] = mix_markovs_over_depths_dims[depth].cum_loss

add_ctws_over_dims_cum_loss = dict()
for depth in add_ctws_over_dims.keys():
    add_ctws_over_dims_cum_loss[depth] = add_ctws_over_dims[depth].cum_loss

mix_ctws_over_dims_cum_loss = dict()
for depth in mix_ctws_over_dims.keys():
    mix_ctws_over_dims_cum_loss[depth] = mix_ctws_over_dims[depth].cum_loss


In [ ]:
import shelve

filename = "results/{}-cum-losses.out".format(dataset.name)
my_shelf = shelve.open(filename, 'n') # 'n' for new

var_names = [
    'dataset_name',
    'dim',
    'ogds_cum_loss',
    'kt_markovs_cum_loss',
    'dfeg_markovs_cum_loss',
    'adanorm_markovs_cum_loss',
    'ctws_cum_loss',
    'add_markovs_over_dims_cum_loss',
    'mix_markovs_over_dims_cum_loss',
    'add_markovs_over_depths_dims_cum_loss',
    'mix_markovs_over_depths_dims_cum_loss',
    'add_ctws_over_dims_cum_loss',
    'mix_ctws_over_dims_cum_loss',
]

for key in var_names:
    try:
        my_shelf[key] = globals()[key]
    except TypeError:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
my_shelf.close()